# Curl 轉 Requests 轉換指南

## 基本概念

`curl` 是一個命令行工具，而 `requests` 是 Python 的 HTTP 庫。以下是常見的轉換模式：

## 1. GET 請求

### Curl
```bash
curl "https://api.example.com/users"
```

### Requests
```python
import requests

response = requests.get("https://api.example.com/users")
print(response.json())
```

## 2. POST 請求 (JSON 數據)

### Curl
```bash
curl -X POST "https://api.example.com/users" \
     -H "Content-Type: application/json" \
     -d '{"name": "John", "email": "john@example.com"}'
```

### Requests
```python
import requests

url = "https://api.example.com/users"
data = {
    "name": "John",
    "email": "john@example.com"
}

response = requests.post(url, json=data)
print(response.json())
```

## 3. 帶有 Headers 的請求

### Curl
```bash
curl -X GET "https://api.example.com/protected" \
     -H "Authorization: Bearer your-token-here" \
     -H "Content-Type: application/json"
```

### Requests
```python
import requests

url = "https://api.example.com/protected"
headers = {
    "Authorization": "Bearer your-token-here",
    "Content-Type": "application/json"
}

response = requests.get(url, headers=headers)
print(response.json())
```

## 4. 帶有查詢參數的請求

### Curl
```bash
curl "https://api.example.com/search?q=python&limit=10"
```

### Requests
```python
import requests

url = "https://api.example.com/search"
params = {
    "q": "python",
    "limit": 10
}

response = requests.get(url, params=params)
print(response.json())
```

## 5. 文件上傳

### Curl
```bash
curl -X POST "https://api.example.com/upload" \
     -F "file=@/path/to/file.txt" \
     -F "description=My file"
```

### Requests
```python
import requests

url = "https://api.example.com/upload"
files = {
    "file": open("/path/to/file.txt", "rb")
}
data = {
    "description": "My file"
}

response = requests.post(url, files=files, data=data)
print(response.json())
```

## 6. 處理響應

### 基本響應處理
```python
import requests

response = requests.get("https://api.example.com/data")

# 檢查狀態碼
if response.status_code == 200:
    print("請求成功")
    data = response.json()
    print(data)
else:
    print(f"請求失敗: {response.status_code}")
    print(response.text)
```

### 錯誤處理
```python
import requests

try:
    response = requests.get("https://api.example.com/data")
    response.raise_for_status()  # 如果狀態碼不是 2xx 會拋出異常
    data = response.json()
    print(data)
except requests.exceptions.RequestException as e:
    print(f"請求發生錯誤: {e}")
```

## 7. 會話管理 (Session)

### Curl (多個請求)
```bash
# 登入
curl -X POST "https://api.example.com/login" \
     -d "username=user&password=pass" \
     -c cookies.txt

# 使用 cookies 的後續請求
curl -X GET "https://api.example.com/profile" \
     -b cookies.txt
```

### Requests (Session)
```python
import requests

# 創建會話
session = requests.Session()

# 登入
login_data = {
    "username": "user",
    "password": "pass"
}
login_response = session.post("https://api.example.com/login", data=login_data)

# 使用同一個會話進行後續請求
profile_response = session.get("https://api.example.com/profile")
print(profile_response.json())
```

## 8. 實際範例：Ollama API

### Curl
```bash
curl -X POST "http://localhost:11434/api/generate" \
     -H "Content-Type: application/json" \
     -d '{
       "model": "gemma2:2b",
       "prompt": "天空為什麼是藍的?",
       "stream": false
     }'
```

### Requests
```python
import requests

url = "http://localhost:11434/api/generate"
data = {
    "model": "gemma2:2b",
    "prompt": "天空為什麼是藍的?",
    "stream": False
}

response = requests.post(url, json=data)
result = response.json()
print(result["response"])
```

## 9. 常用轉換對照表

| Curl 選項 | Requests 參數 | 說明 |
|-----------|---------------|------|
| `-X POST` | `requests.post()` | HTTP 方法 |
| `-H "Header: Value"` | `headers={"Header": "Value"}` | 請求頭 |
| `-d '{"key": "value"}'` | `json={"key": "value"}` | JSON 數據 |
| `-d "key=value"` | `data={"key": "value"}` | 表單數據 |
| `-F "file=@path"` | `files={"file": open("path", "rb")}` | 文件上傳 |
| `-b cookies.txt` | `cookies=requests.cookies.RequestsCookieJar()` | Cookies |
| `--timeout 30` | `timeout=30` | 超時設定 |

## 10. 最佳實踐

1. **總是處理異常**：使用 try-except 包裝請求
2. **檢查狀態碼**：確認請求是否成功
3. **使用 Session**：對於多個相關請求，使用 Session 保持連接
4. **設定超時**：避免請求無限等待
5. **關閉文件**：上傳文件後記得關閉

```python
import requests

def safe_request(url, **kwargs):
    try:
        response = requests.get(url, timeout=30, **kwargs)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"請求失敗: {e}")
        return None

# 使用範例
data = safe_request("https://api.example.com/data")
if data:
    print(data)
```


In [1]:
import requests
import markdown
from IPython.display import HTML, display

url = "http://localhost:11434/api/generate"
data = {
    "model": "gemma3:1b",
    "prompt":"天空為什麼是藍的?",
    "stream": False
}

response = requests.post(url, json=data)
if response.status_code == 200:
    print("請求成功!!")
    answer = response.json()["response"]
    #print(f"AI回答: {answer}")

    #todo 將回答以 Markdown 預覽顯示
    markdown_answer = markdown.markdown(answer)
    #print(f"Markdwon 格式:\n {markdown_answer}")

    #todo 將markdown_answer的HTML格式顯示於螢幕上
    display(HTML(markdown_answer))
else:
    print(f"請求失敗!! 狀態碼: {response.status_code}")
    print("錯誤訊息:", response.text)

KeyboardInterrupt: 

In [ ]:
# 更完整的錯誤處理版本
import requests

def test_ollama_api():
    url = "http://localhost:11434/api/generate"
    data = {
        "model": "gemma3:1b",
        "prompt": "天空為什麼是藍的?",
        "stream": False
    }
    
    try:
        print("正在發送請求到 Ollama API...")
        response = requests.post(url, json=data, timeout=30)
        
        print(f"狀態碼: {response.status_code}")
        
        if response.status_code == 200:
            print("✅ 請求成功!!")
            result = response.json()
            print("完整回應:", result)
            
            # 提取 AI 的回答
            if "response" in result:
                print(f"\n🤖 AI 回答: {result['response']}")
            else:
                print("⚠️ 回應中沒有找到 'response' 欄位")
                
        else:
            print(f"❌ 請求失敗!! 狀態碼: {response.status_code}")
            print("錯誤詳情:", response.text)
            
    except requests.exceptions.ConnectionError:
        print("❌ 連接錯誤: 無法連接到 Ollama 服務")
        print("請確認 Ollama 是否正在運行: ollama serve")
        
    except requests.exceptions.Timeout:
        print("❌ 請求超時: 請檢查網路連接或增加超時時間")
        
    except requests.exceptions.RequestException as e:
        print(f"❌ 請求發生錯誤: {e}")
        
    except Exception as e:
        print(f"❌ 未預期的錯誤: {e}")

# 執行測試
test_ollama_api()
